Install all the required packages

In [1]:
!pip -q install langchain
!pip -q install langchain-community
!pip -q install datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate transformers
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install tiktoken
!pip -q install unstructured
!pip install tokenizers
!pip install xformers
!pip install pinecone-client

  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


Import all the required libraries

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Passing URLs

In [4]:
URLs= [
      'https://sh-tsang.medium.com/brief-review-llama-2-open-foundation-and-fine-tuned-chat-models-6666eb8b56b7',
      'https://www.databricks.com/blog/mpt-7b',
      'https://stability.ai/stable-lm',
      'https://lmsys.org/blog/2023-03-30-vicuna/'
]

In [5]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [6]:
data

[Document(metadata={'source': 'https://sh-tsang.medium.com/brief-review-llama-2-open-foundation-and-fine-tuned-chat-models-6666eb8b56b7'}, page_content='Open in app\n\nSign in\n\nWrite\n\nSign in\n\nBrief Review — Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2, Open Source and Released\n\nSik-Ho Tsang\n\nFollow\n\n4 min read\n\nAug 5, 2023\n\n--\n\nWebsite: https://ai.meta.com/llama/\n\nLanguage Model 1991 … 2023 [GPT-4] [LLaMA] [LIMA] [Koala] [BloombergGPT] [GLM-130B] [UL2] [PaLM 2] ==== My Other Paper Readings Are Also Over Here ====\n\nLlama 2: Open Foundation and Fine-Tuned Chat Models Llama 2, by GenAI, Meta, 2023 arXiv v2 (Sik-Ho Tsang @ Medium)\n\nLlama 2 is developed and released, ranging in scale from 7 billion to 70 billion parameters.\n\nThe fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. These Llama 2 models outperform open-source chat models on most benchmarks, and based on human evaluations for helpfulness and safety, may be a s

In [7]:
len(data)

4

Split the Text into Chunks


In [8]:
text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)

In [9]:
text_chunks = text_splitter.split_documents(data)

In [10]:
len(text_chunks)

61

In [11]:
text_chunks[0]

Document(metadata={'source': 'https://sh-tsang.medium.com/brief-review-llama-2-open-foundation-and-fine-tuned-chat-models-6666eb8b56b7'}, page_content='Open in app\nSign in\nWrite\nSign in\nBrief Review — Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2, Open Source and Released\nSik-Ho Tsang\nFollow\n4 min read\nAug 5, 2023\n--\nWebsite: https://ai.meta.com/llama/\nLanguage Model 1991 … 2023 [GPT-4] [LLaMA] [LIMA] [Koala] [BloombergGPT] [GLM-130B] [UL2] [PaLM 2] ==== My Other Paper Readings Are Also Over Here ====\nLlama 2: Open Foundation and Fine-Tuned Chat Models Llama 2, by GenAI, Meta, 2023 arXiv v2 (Sik-Ho Tsang @ Medium)\nLlama 2 is developed and released, ranging in scale from 7 billion to 70 billion parameters.\nThe fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. These Llama 2 models outperform open-source chat models on most benchmarks, and based on human evaluations for helpfulness and safety, may be a suitable substitute for closed-s

In [21]:
text_chunks[1]

Document(metadata={'source': 'https://sh-tsang.medium.com/brief-review-llama-2-open-foundation-and-fine-tuned-chat-models-6666eb8b56b7'}, page_content='Outline\nLlama 2: Pretraining\nLlama 2-Chat: Fine-Tuning\n1. Llama 2: Pretraining\n1.1. Models & Data\nLlama 2 models use the pretraining approach in Llama 1, using an optimized auto-regressive transformer\nEnhanced techniques are used: More robust data cleaning, updated data mixes, trained on 40% more total tokens, doubled the context length, and used grouped-query attention (GQA) to improve inference scalability for the larger models. The primary architectural differences from Llama 1 include increased context length and GQA.\nThe training corpus includes a new mix of data from publicly available sources, which does not include data from Meta’s products or services.\nThe models are trained using 2 trillion tokens of data as this provides a good performance–cost trade-off, up-sampling the most factual sources in an effort to increase k

Download the hugging face embeddings


In [12]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-12-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-12-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

768

In [15]:
query_result

[0.026249706745147705,
 0.013395607471466064,
 -0.004533133003860712,
 -0.021791422739624977,
 0.05455183610320091,
 -0.004966470878571272,
 0.006655557081103325,
 0.03062625601887703,
 -0.005762824323028326,
 -0.004562034271657467,
 -0.003313301596790552,
 -0.0484962984919548,
 -0.011364065110683441,
 0.03507744148373604,
 0.09309469908475876,
 -0.0866873636841774,
 0.051086537539958954,
 0.009886160492897034,
 -0.06356934458017349,
 -0.008550204336643219,
 0.007054392248392105,
 -0.003862351179122925,
 0.024744288995862007,
 0.04288491606712341,
 0.035094160586595535,
 -0.02984827198088169,
 0.010252618230879307,
 0.02234490029513836,
 0.020889993757009506,
 0.00949221383780241,
 -0.03304434195160866,
 -0.012284105643630028,
 0.05352889373898506,
 0.02542915940284729,
 2.022177341132192e-06,
 -0.034191031008958817,
 0.009610011242330074,
 -0.016484549269080162,
 0.005609536077827215,
 -0.004250032361596823,
 -0.022801222279667854,
 0.04035475477576256,
 0.0030519880820065737,
 0.0313

Convert the text chunks into embeddings and create a knowledge base

In [26]:
# Assuming `text_chunks` is a list of Document objects
document_dicts = [
    {
        "id": f"{d.metadata.get('source', 'unknown_source')}_page_{d.metadata.get('page', 'unknown_page')}",  # Use .get() to avoid KeyError
        "text": d.page_content  # The actual content of the document chunk
    }
    for d in text_chunks  # Iterate over each Document object in text_chunks
]

# Check the result
print(document_dicts[:2])  # Print the first two to check


[{'id': 'https://sh-tsang.medium.com/brief-review-llama-2-open-foundation-and-fine-tuned-chat-models-6666eb8b56b7_page_unknown_page', 'text': 'Open in app\nSign in\nWrite\nSign in\nBrief Review — Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2, Open Source and Released\nSik-Ho Tsang\nFollow\n4 min read\nAug 5, 2023\n--\nWebsite: https://ai.meta.com/llama/\nLanguage Model 1991 … 2023 [GPT-4] [LLaMA] [LIMA] [Koala] [BloombergGPT] [GLM-130B] [UL2] [PaLM 2] ==== My Other Paper Readings Are Also Over Here ====\nLlama 2: Open Foundation and Fine-Tuned Chat Models Llama 2, by GenAI, Meta, 2023 arXiv v2 (Sik-Ho Tsang @ Medium)\nLlama 2 is developed and released, ranging in scale from 7 billion to 70 billion parameters.\nThe fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. These Llama 2 models outperform open-source chat models on most benchmarks, and based on human evaluations for helpfulness and safety, may be a suitable substitute for closed-source mod

In [24]:
pip install "pinecone[grpc]"

In [ ]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="***********************************")


# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in document_dicts],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings)




EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [0.023406982421875, 0.00039315223693847656, ..., -0.024993896484375, -0.0080108642578125]},
    {'vector_type': dense, 'values': [0.0100860595703125, -0.00484466552734375, ..., -0.01580810546875, -0.004608154296875]},
    ... (57 more embeddings) ...,
    {'vector_type': dense, 'values': [0.00348663330078125, -0.008697509765625, ..., -0.0086822509765625, -0.00681304931640625]},
    {'vector_type': dense, 'values': [0.0222320556640625, 0.00223541259765625, ..., -0.0266265869140625, -0.0015354156494140625]}
  ],
  usage={'total_tokens': 12931}
)


In [34]:
# Target the index where you'll store the vector embeddings
index = pc.Index("websitechatbot")

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for d, e in zip(document_dicts, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace"
)


upserted_count: 61

Create a large language model wrapper


In [36]:
notebook_login()

In [37]:
model="daryl149/llama-2-7b-chat-hf"

In [38]:
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(model,
                                             load_in_8bit=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [40]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0


In [41]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

<ipython-input-41-9753c688c802>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [42]:
llm.predict("Please provide a summary of the MPT-7B")

<ipython-input-42-e7ca5e4dfd07>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a summary of the MPT-7B")


'Please provide a summary of the MPT-7B tablet’s features and functionalities.\nHere is a summary of the MPT-7B tablet’s features and functionalities:\nOS: Android 10.0 (Go Edition)\nMemory: 2GB RAM, 32GB internal storage (expandable up to 128GB via microSD card)\nDisplay: 7-inch IPS HD display with a resolution of 1280 x 800 pixels\nCamera: 5MP rear camera and 2MP front camera\nBattery: 5,000mAh battery with up to 8 hours of battery life\nConnectivity: Wi-Fi, Bluetooth 4.2, and Micro-USB port\nSensors: Accelerometer, ambient light sensor, and proximity sensor\nDimensions: 178.8 x 105.8 x 11.4 mm\nWeight: 345 grams\nThese are the main features and functionalities of the MPT-7B tablet.'

then you can initialize the retreival qa with source chain